# phase 1 分完重點後，切換到對應的 thread

In [ ]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
import pandas as pd
import numpy as np

csv_file = './data/key_point_embeddings0527.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

In [ ]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [ ]:
def get_similar_QAK(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    key_point = np.array(response.data[0].embedding, ndmin=2)
    distances, indices = index.search(key_point, 3)
    QAK_csv = './data/QAK0527funver.csv'
    QAK_df = pd.read_csv(QAK_csv)
    QAK_array = QAK_df.to_numpy()
    QAKD = [(QAK_array[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    top3 = ""
    for i in range(0,3):
        top3 = top3 + '{"problem": "' + QAKD[i][0][0].replace("\"", "\'") + '",\n'
        top3 = top3 + '"summarized keypoints": ["' + QAKD[i][0][2][16:-3].replace("\"", "\"") + '"],\n'
        top3 = top3 + '"solution": { \n "form": "'+QAKD[i][0][3]+'", \n "content": "' + QAKD[i][0][1].replace("\"", "\'") + '"}},\n'
    top3 = top3[0:-2] + ']'
    return top3

def get_similar_flow(input):
    pass

# 模擬 phase1

In [ ]:
question = "My total time is wrong"

In [ ]:
instruction = """
    # Objective:
    As an intelligent customer service chatbot of the “Flora” focus app , my goal is to understand and help you solve problems encountered when using flora app effectively.
    # Main features of the Flora app:
    Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what’s more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track.* * Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most.
    # Support Process:
    Issue Identification: To ensure that I understand your problems, I will summarize the constructive, non-emotional parts of our input text in your tone into one key point. but if one key point is not enough to describe problem, key point can be more than one. Avoid using conjunctions. Then, I will and must only output these key points in json format.
    # Example:
    input: How do I change my password?
    output: {"keypoints": ["my problem is I need to change password."]}
    input: I am just doing my a-levels at the moment and I love using your app for revision. However, when I went onto the app earlier it logged me out and all my data had been lost. I was just wondering if you could help me out.
    output: {"keypoints": ["My data lost after being logged out of the app."]}
    input: My flora friend page won't load. If I delete the app and re download it and then login. Will all my data and garden time be lost?
    output: {"keypoints": ["my Flora friend page will not load.","I am concern about losing data and garden time after deleting and reinstalling the app."]}
"""

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": question}
    ]
)
time.sleep(3)
key_point_json = completion.choices[0].message.content.replace("\n", " ")
print(key_point_json)

In [ ]:
import json

key_point = []
try:
    for i in range(0,len(json.loads(key_point_json)["keypoints"])):
        key_point.append(str(json.loads(key_point_json)["keypoints"][i]))

except json.JSONDecodeError as e:
    key_point = key_point_json
    print("phase1 didn't pass json format")

print(key_point)

# 模擬 phase2

In [ ]:
add_hawaii_record_flag = 0
correct_weekly_leaderboard_flag = 0
check_total_time_flag = 0

def external_add_hawaii_record(userID):
    global add_hawaii_record_flag
    add_hawaii_record_flag = 1
    return "We add Hawaii record for you, thank you for your patience."

def external_correct_weekly_leaderboard(userID):
    global correct_weekly_leaderboard_flag
    correct_weekly_leaderboard_flag = 1
    return "We update your WeeklyLeaderboard, thank you for your patience."

def external_check_total_time(userID):
    global check_total_time_flag
    check_total_time_flag = 1
    return "We recalculate your Total time, thank you for your patience. Is this your total time ?"

In [ ]:
threadNum = 0

instruction = """
{
    "objective": "Act as an intelligent customer service chatbot for the 'Flora' focus app.",
    "job": [
        "1. Decide which candidate problem-resolution approach can be applied to solve the user's main problem. Note that the conversation log may contain irrelevant information and you should only use it as a clue for solving the user's main problem. First, if user's main problem is not related to Flora, you are prohibited to answer and must reply only with a special mark: '*'. Second, if the summarized key points within the three problem-resolution approaches are not related to the user's main problem, you are prohibited from answering and must reply only with a special mark: '*'. Third, if there are multiple related approaches, choose the one that is the most suitable.",
        "2. Based on the chosen problem-resolution approach, chat with user to solve the main problem without any format. If the approach contains an answer directly, follow the answer to interact with the user. You must not deviate from the chat based on the answer. If the approach contains a function, execute the exact function and continue to interact with the user.",
        "3. If the user asks another question different from the user's main problem either during or after the problem resolution process in Step 2, prompt the user to go back to the 'Feedback Portal' system to officially file the problem."
    ],
    "Main features of the Flora app": "Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what's more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track. Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most."
}
"""
print(instruction)

In [ ]:
assistant = client.beta.assistants.create(
    name = "customer service chatbot",
    instructions = instruction,
    model = "gpt-3.5-turbo-1106",
    tools = [
        {
            "type": "function",
            "function": {
                "name": "function_1",
                "description": "If the chosen problem-resolution approach contain the function: function_1, This function must be called. The return of function_1 is a workflow in the form of pseudo code and you must follow.",
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_1",
                "description": "This function must be called by function_1 function.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user_ID": {
                            "type": "string",
                            "description": "user's ID in Flora app. This must be gained from asking the user.",
                        },
                    },
                    "required": ["user_ID"],
                },
            },
        },
        # {
        #     "type": "function",
        #     "function": {
        #         "name": "correct_Tour_Progress",
        #         "description": "If the chosen problem-resolution approach contain the function: correct_Tour_Progress, This function must be called. The return of correct_Tour_Progress is a workflow in the form of pseudo code and you must follow.",
        #     },
        # },
        {
            "type": "function",
            "function": {
                "name": "function_3",
                "description": "If the chosen problem-resolution approach contain the function: function_3, This function must be called. The return of function_3 is a workflow in the form of pseudo code and you must follow."
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_3",
                "description": "This function must be called by function_3 function.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user_ID": {
                            "type": "string",
                            "description": "user's ID in Flora app. This must be gained from asking the user.",
                        },
                    },
                    "required": ["user_ID"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "function_4",
                "description": "If the chosen problem-resolution approach contain the function: function_4, This function must be called. The return of function_4 is a workflow in the form of pseudo code and you must follow."
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_4",
                "description": "This function must be called by function_4 function.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user_ID": {
                            "type": "string",
                            "description": "user's ID in Flora app. This must be gained from asking the user.",
                        },
                    },
                    "required": ["user_ID"],
                },
            },
        },
    ]
)

thread = client.beta.threads.create()

In [ ]:
user_prompt="{"

conversation_log = '"Past Conversation Log": [{"role": "user", "message": "' + question + '"}, {"role": "assistant", "message": "This is my summarized key points:'+ str(key_point).replace('"',"'") + ', and i will focus on main problem and help you solve it."}],'
user_prompt = user_prompt + conversation_log + "\n"

main_problem = key_point[threadNum]
user_question ='"Main Problem": "'
user_prompt = user_prompt + user_question + main_problem + '",\n'

candidates = '"Candidate Problem-resolution Approaches": [\n'
top3 = get_similar_QAK(main_problem)
candidates = candidates + top3
user_prompt += candidates

user_prompt += "}"

print(user_prompt)

In [ ]:
run_tools = []

instruction_json = json.loads(user_prompt)
for candidates in instruction_json["Candidate Problem-resolution Approaches"]:
    if candidates["solution"]["content"] == "function_1":
        function_1 = {
            "type": "function",
            "function": {
                "name": "function_1",
                "description": "If the chosen problem-resolution approach contain the function: function_1, This function must be called. The return of function_1 is a workflow in the form of pseudo code and you must follow.",
            },
        }
        if not (function_1 in run_tools):
            run_tools.append(function_1)

    # if candidates["solution"]["content"] == "function_2":
    #     run_tools.append()

    if candidates["solution"]["content"] == "function_3":
        function_3 = {
            "type": "function",
            "function": {
                "name": "function_3",
                "description": "If the chosen problem-resolution approach contain the function: function_3, This function must be called. The return of function_3 is a workflow in the form of pseudo code and you must follow."
            },
        }
        if not (function_3 in run_tools):
            run_tools.append(function_3)

    if candidates["solution"]["content"] == "function_4":
        function_4 = {
            "type": "function",
            "function": {
                "name": "function_4",
                "description": "If the chosen problem-resolution approach contain the function: function_4, This function must be called. The return of function_4 is a workflow in the form of pseudo code and you must follow."
            },
        }
        if not (function_4 in run_tools):
            run_tools.append(function_4)

for tools in run_tools:
    print(tools)

In [ ]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = user_prompt
)

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    tools = run_tools,
    parallel_tool_calls = False
)
while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
    
print(run.status)
if run.status == "requires_action":
    for func in run.required_action.submit_tool_outputs.tool_calls:
        print(func)

In [ ]:
if run.status == "requires_action":
    toolOutput = []
    for func in run.required_action.submit_tool_outputs.tool_calls:
        # if func.function.name == "ask_id":
        #     flow = "Ask user for their ID; if(get user ID){execute next tool calling;} else{Ask user ID again;}"
        #     tmp = {"tool_call_id": func.id, "output": flow}
        #     toolOutput.append(tmp)

        if func.function.name == "function_1":
            flow = "Ask user for their ID; if(get user ID){call external_1 function;} else{Ask user ID again;}"
            tmp = {"tool_call_id": func.id, "output": flow}
            toolOutput.append(tmp)
        if func.function.name == "external_1":
            user_id = eval(func.function.arguments)["user_ID"]
            result = external_add_hawaii_record(user_id)
            tmp = {"tool_call_id": func.id, "output": result}
            toolOutput.append(tmp)

        ## todo  correct_Tour_Progress

        if func.function.name == "function_3":
            flow = "Ask user for their ID; if(get user ID){call external_3 function;} else{Ask user ID again;}"
            tmp = {"tool_call_id": func.id, "output": flow}
            toolOutput.append(tmp)
        if func.function.name == "external_3":
            user_id = eval(func.function.arguments)["user_ID"]
            result = external_correct_weekly_leaderboard(user_id)
            tmp = {"tool_call_id": func.id, "output": result}
            toolOutput.append(tmp)

        if func.function.name == "function_4":
            flow = "Ask user for their ID; if(get user ID){call external_4 function;} else{Ask user ID again;}"
            tmp = {"tool_call_id": func.id, "output": flow}
            toolOutput.append(tmp)
        if func.function.name == "external_4":
            user_id = eval(func.function.arguments)["user_ID"]
            result = external_check_total_time(user_id)
            tmp = {"tool_call_id": func.id, "output": result}
            toolOutput.append(tmp)

    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id = thread.id,
        run_id = run.id,
        tool_outputs = toolOutput
    )
else:
    print("No function calling")

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

user_or_assistant = 0
for i in messages.data[-5:]:
    if user_or_assistant == 0:
        user_or_assistant = 1
        print("User: ")
    elif user_or_assistant == 1:
        user_or_assistant = 0
        print("Assistant: ")
    print(i.content[0].text.value)
    print("\n")

# 後續對話

In [ ]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "abc12345"
    # content = "thanks. By the way, can you recommed some App that can help me improve my Emglish ability?"
    # content = "how to get Hawaii tour?"
)

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    parallel_tool_calls = False
)
while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
print(run.status)
if run.status == "requires_action":
    for func in run.required_action.submit_tool_outputs.tool_calls:
        print(func)

In [ ]:
if run.status == "requires_action":
    toolOutput = []
    for func in run.required_action.submit_tool_outputs.tool_calls:
        # if func.function.name == "ask_id":
        #     flow = "Ask user for their ID; if(get user ID){execute next tool calling;} else{Ask user ID again;}"
        #     tmp = {"tool_call_id": func.id, "output": flow}
        #     toolOutput.append(tmp)

        if func.function.name == "function_1":
            flow = "Ask user for their ID; if(get user ID){call external_1 function;} else{Ask user ID again;}"
            tmp = {"tool_call_id": func.id, "output": flow}
            toolOutput.append(tmp)
        if func.function.name == "external_1":
            user_id = eval(func.function.arguments)["user_ID"]
            result = external_add_hawaii_record(user_id)
            tmp = {"tool_call_id": func.id, "output": result}
            toolOutput.append(tmp)

        ## todo  correct_Tour_Progress

        if func.function.name == "function_3":
            flow = "Ask user for their ID; if(get user ID){call external_3 function;} else{Ask user ID again;}"
            tmp = {"tool_call_id": func.id, "output": flow}
            toolOutput.append(tmp)
        if func.function.name == "external_3":
            user_id = eval(func.function.arguments)["user_ID"]
            result = external_correct_weekly_leaderboard(user_id)
            tmp = {"tool_call_id": func.id, "output": result}
            toolOutput.append(tmp)

        if func.function.name == "function_4":
            flow = "Ask user for their ID; if(get user ID){call external_4 function;} else{Ask user ID again;}"
            tmp = {"tool_call_id": func.id, "output": flow}
            toolOutput.append(tmp)
        if func.function.name == "external_4":
            user_id = eval(func.function.arguments)["user_ID"]
            result = external_check_total_time(user_id)
            tmp = {"tool_call_id": func.id, "output": result}
            toolOutput.append(tmp)

    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id = thread.id,
        run_id = run.id,
        tool_outputs = toolOutput
    )
else:
    print("No function calling")

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

user_or_assistant = 0
for i in messages.data[-10:]:
    if user_or_assistant == 0:
        user_or_assistant = 1
        print("User: ")
    elif user_or_assistant == 1:
        user_or_assistant = 0
        print("Assistant: ")
    print(i.content[0].text.value)
    print("\n")

In [ ]:
print("add_hawaii_record_flag: ", add_hawaii_record_flag)
print("correct_Weekly_Leaderboard_flag: ", correct_weekly_leaderboard_flag)
print("correct_Total_Time_flag: " , check_total_time_flag)

# Delete Assistant

In [ ]:
client.beta.assistants.delete(assistant.id)

In [ ]:
client.beta.assistants.list(
    order="desc",
    limit="20",
)